Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls. 

1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)

In [1]:
#! pip install pulp

In [2]:
from pulp import *

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('diet.csv')
#df.head()
df.tail()
#Notice the last row of the data set was actually row 63.(WhicH we could also see that when open the excel when first download the data)
#then to only extrac the data, only need until row 63.
food=df[0:64]

In [5]:
food.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,$0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",$0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",$0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,$0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",$0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [6]:
#food.describe()

In [7]:
food['Price/ Serving']=food['Price/ Serving'].astype(str)

C:\Users\CHAOS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
#since convert cost to float, we will be 
food['Price/ Serving'] = food['Price/ Serving'].str.replace('$', '')

C:\Users\CHAOS\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
food.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [10]:
foodlist=food.values.tolist()

In [11]:
#food items
item=[x[0] for x in foodlist]

In [12]:
cost=dict([(x[0],float(x[1])) for x in foodlist])

In [13]:
cal=dict([(x[0],float(x[3])) for x in foodlist])

In [14]:
clt=dict([(x[0],float(x[4])) for x in foodlist])

In [15]:
ttf=dict([(x[0],float(x[5])) for x in foodlist])

In [16]:
sd=dict([(x[0],float(x[6])) for x in foodlist])

In [17]:
carb=dict([(x[0],float(x[7])) for x in foodlist])

In [18]:
fiber=dict([(x[0],float(x[8])) for x in foodlist])

In [19]:
po =dict([(x[0],float(x[9])) for x in foodlist])

In [20]:
va=dict([(x[0],float(x[10])) for x in foodlist])

In [21]:
vc=dict([(x[0],float(x[11])) for x in foodlist])

In [22]:
cci=dict([(x[0],float(x[12])) for x in foodlist])

In [23]:
iron=dict([(x[0],float(x[13])) for x in foodlist])

In [24]:
#defined a problem name
#Minimizing cost
problem = LpProblem('mealplan', LpMinimize)

In [25]:
#define variable
amountVars = LpVariable.dicts('Amount',item, 0)

In [26]:
#Add  objective function to the problem
problem += lpSum([cost[i]* amountVars[i] for i in item]), 'total cost'

In [27]:
#constraints hard code on each ones
#calories
problem += lpSum([cal[i]* amountVars[i] for i in item])>=1500, 'min cal'
problem += lpSum([cal[i]* amountVars[i] for i in item])<=2500, 'max cal'
#cholesterol
problem += lpSum([clt[i]* amountVars[i] for i in item])>=30, 'min cholesterol'
problem += lpSum([clt[i]* amountVars[i] for i in item])<=240, 'max cholesterol'

#fat
problem += lpSum([ttf[i]* amountVars[i] for i in item])>=20, 'min fat'
problem += lpSum([ttf[i]* amountVars[i] for i in item])<=70, 'max fat'

#sodium
problem += lpSum([sd[i]* amountVars[i] for i in item])>=800, 'min sodium'
problem += lpSum([sd[i]* amountVars[i] for i in item])<=2000, 'max sodium'
#carb
problem += lpSum([carb[i]* amountVars[i] for i in item])>=130, 'min carb'
problem += lpSum([carb[i]* amountVars[i] for i in item])<=450, 'max carb'

#fiber
problem += lpSum([fiber[i]* amountVars[i] for i in item])>=125, 'min fiber'
problem += lpSum([fiber[i]* amountVars[i] for i in item])<=250, 'max fiber'

#protein
problem += lpSum([po[i]* amountVars[i] for i in item])>=60, 'min protein'
problem += lpSum([po[i]* amountVars[i] for i in item])<=100, 'max protein'

#va
problem += lpSum([va[i]* amountVars[i] for i in item])>=1000, 'min vitaminA'
problem += lpSum([va[i]* amountVars[i] for i in item])<=10000, 'max vitaminA'

#vc
problem += lpSum([vc[i]* amountVars[i] for i in item])>=400, 'min vitaminC'
problem += lpSum([vc[i]* amountVars[i] for i in item])<=5000, 'max vitaminC'

#cci
problem += lpSum([cci[i]* amountVars[i] for i in item])>=700, 'min calcium'
problem += lpSum([cci[i]* amountVars[i] for i in item])<=1500, 'max calcium'

#iron
problem += lpSum([iron[i]* amountVars[i] for i in item])>=10, 'min iron'
problem += lpSum([iron[i]* amountVars[i] for i in item])<=40, 'max iron'

In [28]:
problem.solve()

1

In [29]:
varsDictionary = {}
for v in problem.variables():
    if v.varValue >0.0:
        
        varsDictionary[v.name] = v.varValue
print(varsDictionary)

{'Amount_Celery,_Raw': 52.64371, 'Amount_Frozen_Broccoli': 0.25960653, 'Amount_Lettuce,Iceberg,Raw': 63.988506, 'Amount_Oranges': 2.2929389, 'Amount_Poached_Eggs': 0.14184397, 'Amount_Popcorn,Air_Popped': 13.869322}


In [30]:
print ("Total Cost of Foods per person = ", value(problem.objective))

Total Cost of Foods per person =  4.337116797399999


2.	Please add to your model the following constraints (which might require adding more variables) and solve the new model:
a.	If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
b.	Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
c.	To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]

In [31]:
#amountVars = LpVariable.dicts('Amount',item, 0)
#a.Hint: now you will need two variables for each food i:
#whether it is chosen,and how much is part of the diet. 
#You’ll also need to write a constraint to link them.) 
food_choice=LpVariable.dicts('foodchosen', item, 0,1,LpBinary) 
#define variable
amountVars = LpVariable.dicts('Amount',item, 0)
#Add  objective function to the problem
problem_cons = LpProblem('MealPlan2', LpMinimize)
#
problem_cons += lpSum([cost[i] * amountVars[i] for i in item]), 'Total Cost'



In [32]:
food_choice

{'Frozen Broccoli': foodchosen_Frozen_Broccoli,
 'Carrots,Raw': foodchosen_Carrots,Raw,
 'Celery, Raw': foodchosen_Celery,_Raw,
 'Frozen Corn': foodchosen_Frozen_Corn,
 'Lettuce,Iceberg,Raw': foodchosen_Lettuce,Iceberg,Raw,
 'Peppers, Sweet, Raw': foodchosen_Peppers,_Sweet,_Raw,
 'Potatoes, Baked': foodchosen_Potatoes,_Baked,
 'Tofu': foodchosen_Tofu,
 'Roasted Chicken': foodchosen_Roasted_Chicken,
 'Spaghetti W/ Sauce': foodchosen_Spaghetti_W__Sauce,
 'Tomato,Red,Ripe,Raw': foodchosen_Tomato,Red,Ripe,Raw,
 'Apple,Raw,W/Skin': foodchosen_Apple,Raw,W_Skin,
 'Banana': foodchosen_Banana,
 'Grapes': foodchosen_Grapes,
 'Kiwifruit,Raw,Fresh': foodchosen_Kiwifruit,Raw,Fresh,
 'Oranges': foodchosen_Oranges,
 'Bagels': foodchosen_Bagels,
 'Wheat Bread': foodchosen_Wheat_Bread,
 'White Bread': foodchosen_White_Bread,
 'Oatmeal Cookies': foodchosen_Oatmeal_Cookies,
 'Apple Pie': foodchosen_Apple_Pie,
 'Chocolate Chip Cookies': foodchosen_Chocolate_Chip_Cookies,
 'Butter,Regular': foodchosen_Butt

In [33]:
# a. If a food is selected, then a minimum of 1/10 serving must be chosen
for i in item:
    problem_cons += amountVars[i]>= food_choice[i]*0.1

for i in item:
    problem_cons += food_choice[i]>= amountVars[i]*0.0000001

In [34]:
#constraints hard code on each ones
#calories
problem_cons += lpSum([cal[i]* amountVars[i] for i in item])>=1500, 'min cal'
problem_cons += lpSum([cal[i]* amountVars[i] for i in item])<=2500, 'max cal'
#cholesterol
problem_cons += lpSum([clt[i]* amountVars[i] for i in item])>=30, 'min cholesterol'
problem_cons += lpSum([clt[i]* amountVars[i] for i in item])<=240, 'max cholesterol'

#fat
problem_cons += lpSum([ttf[i]* amountVars[i] for i in item])>=20, 'min fat'
problem_cons += lpSum([ttf[i]* amountVars[i] for i in item])<=70, 'max fat'

#sodium
problem_cons += lpSum([sd[i]* amountVars[i] for i in item])>=800, 'min sodium'
problem_cons += lpSum([sd[i]* amountVars[i] for i in item])<=2000, 'max sodium'
#carb
problem_cons += lpSum([carb[i]* amountVars[i] for i in item])>=130, 'min carb'
problem_cons += lpSum([carb[i]* amountVars[i] for i in item])<=450, 'max carb'

#fiber
problem_cons += lpSum([fiber[i]* amountVars[i] for i in item])>=125, 'min fiber'
problem_cons += lpSum([fiber[i]* amountVars[i] for i in item])<=250, 'max fiber'

#protein
problem_cons += lpSum([po[i]* amountVars[i] for i in item])>=60, 'min protein'
problem_cons += lpSum([po[i]* amountVars[i] for i in item])<=100, 'max protein'

#va
problem_cons += lpSum([va[i]* amountVars[i] for i in item])>=1000, 'min vitaminA'
problem_cons += lpSum([va[i]* amountVars[i] for i in item])<=10000, 'max vitaminA'

#vc
problem_cons += lpSum([vc[i]* amountVars[i] for i in item])>=400, 'min vitaminC'
problem_cons += lpSum([vc[i]* amountVars[i] for i in item])<=5000, 'max vitaminC'

#cci
problem_cons += lpSum([cci[i]* amountVars[i] for i in item])>=700, 'min calcium'
problem_cons += lpSum([cci[i]* amountVars[i] for i in item])<=1500, 'max calcium'

#iron
problem_cons += lpSum([iron[i]* amountVars[i] for i in item])>=10, 'min iron'
problem_cons += lpSum([iron[i]* amountVars[i] for i in item])<=40, 'max iron'

In [35]:
problem_cons += food_choice['Frozen Broccoli'] + food_choice['Celery, Raw'] <=1

In [36]:
problem_cons += food_choice['Roasted Chicken']+food_choice['Poached Eggs']+food_choice['Scrambled Eggs']\
                +food_choice['Bologna,Turkey']+food_choice['Frankfurter, Beef']+food_choice['Ham,Sliced,Extralean']\
                +food_choice['Kielbasa,Prk']+food_choice['Hamburger W/Toppings']+food_choice['Hotdog, Plain']\
                +food_choice['Macaroni,Ckd'] +food_choice['Pork']+food_choice['Sardines in Oil']\
                +food_choice['White Tuna in Water']+food_choice['Vegetbeef Soup'] >=3
           


In [37]:
problem_cons.solve()

1

In [38]:
varsDictionary_cons = {}
for v in problem_cons.variables():
    if v.varValue >0.0:
        
        varsDictionary_cons[v.name] = v.varValue
print(varsDictionary_cons)

{'Amount_Bologna,Turkey': 0.1, 'Amount_Celery,_Raw': 42.423026, 'Amount_Lettuce,Iceberg,Raw': 82.673927, 'Amount_Oranges': 3.0856009, 'Amount_Peanut_Butter': 1.9590978, 'Amount_Poached_Eggs': 0.1, 'Amount_Popcorn,Air_Popped': 13.214473, 'Amount_Scrambled_Eggs': 0.1, 'foodchosen_Bologna,Turkey': 1.0, 'foodchosen_Celery,_Raw': 1.0, 'foodchosen_Lettuce,Iceberg,Raw': 1.0, 'foodchosen_Oranges': 1.0, 'foodchosen_Peanut_Butter': 1.0, 'foodchosen_Poached_Eggs': 1.0, 'foodchosen_Popcorn,Air_Popped': 1.0, 'foodchosen_Scrambled_Eggs': 1.0}


In [39]:
print("Total cost of food = $%.2f" % value(problem_cons.objective))

Total cost of food = $4.51
